# Загрузка Pandas и очистка данных

In [12]:
import pandas as pd
import numpy as np

In [13]:
unit3 = pd.read_csv('main_task_new.csv')

In [15]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

unit3_numeric = unit3.select_dtypes(include=[np.number])
numeric_cols = unit3_numeric.columns.values

unit3_non_numeric = unit3.select_dtypes(exclude=[np.number])
non_numeric_cols = unit3_non_numeric.columns.values

for col in non_numeric_cols:
    missing = unit3[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        #print('imputing missing values for: {}'.format(col))
        #unit3['{}_ismissing'.format(col)] = missing
        
        top = unit3[col].describe()['top'] # impute with the most frequent value.
        unit3[col] = unit3[col].fillna(top)

for col in numeric_cols:
    missing = unit3[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        #print('imputing missing values for: {}'.format(col))
        #unit3['{}_ismissing'.format(col)] = missing        
        
        med = unit3[col].median()
        unit3[col] = unit3[col].fillna(med)
        
        
cols_to_drop = ['Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA']
unit3_drop_cols = unit3.drop(cols_to_drop, axis=1)


#print(unit3.columns)
#unit3_drop_cols.info()

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [24]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = unit3.drop(['Restaurant_id', 'Rating'], axis = 1)
#X = unit3_drop_cols
y = unit3['Rating']

In [25]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [26]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [27]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [28]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

ValueError: could not convert string to float: 'Lyon'

In [29]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.7041
